In [ ]:
# check out list words and the apostrophe situation in the step for loop
# check out allowing more than maxseqlen in generate text
# look into batch size
#text generation apostrpphe breaking

In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import RMSprop
from keras import optimizers
import sys
from keras.callbacks import LambdaCallback
import random
import matplotlib.pyplot as plt



/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [4]:
def generate_text(seed_text, numb_next_words):
    output=seed_text
    for i in range (numb_next_words):
        
        words_gen = set(seed_text.split())
        words_gen=list(words_gen) #create list of unique words in seed text
        
        
#         for i in range (len(words_gen)): #replace all ' in seed text
#             words_gen[i]=words_gen[i].replace("‘", '').replace("’", '').replace("'", '')
            
        #create a dictionary with index and word
        word_indices_gen = dict((c, i) for i, c in enumerate(words_gen, 1)) 
        
       #turn sentence into a sequence of numbers
        sequence=[] 
        for word in seed_text.split():
            sequence.append(word_indices_gen[word])
        sequence_padded = pad_sequences([sequence], maxlen=10, padding='pre')
#         sequence_padded=sequence
            
        #create an embedding matrix with same indices as word_index 
        EMBEDDING_DIM=25
        total_words=len(word_indices_gen)+1
        embedding_matrix = np.zeros((total_words, EMBEDDING_DIM))
        for word, i in word_indices_gen.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
        #create X input with embedding matrix for specific words (by their index)
        gener=[]
        for number in sequence_padded:
            gener.append(embedding_matrix[number])

        predicted=model.predict([gener], verbose=0)

        predicted=sample(predicted[0])
        output_word=""
        for word, index in word_indices.items():
            if index == predicted:
                output_word = word
                break
        output+=" " + output_word
        seed_text+=" " + output_word
        seed_text=seed_text.split(' ', 1)[1]
    return output

In [5]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(listofwords) - maxlen - 1)
    for diversity in [0.5, 1.0]:
        print('----- diversity:', diversity)
        generated = ''
        sentence = listofwords[start_index: start_index + maxlen].str.cat(sep=' ')
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generate_text(generated, 20))


In [6]:
tweet_data=pd.read_csv('../Load_Tweets/data/tweet_data.csv')

In [7]:
tweet_text = tweet_data['TEXT']

In [8]:
tweet_text_all = tweet_data['TEXT'].str.cat(sep=' ')

In [9]:
listofwords=pd.Series(tweet_text_all.split())

In [10]:
tweet_text.head()

0          PAY TO PLAY POLITICS. #CrookedHillary [URL]
1    Very little pick-up by the dishonest media of ...
2    Crooked Hillary Clinton likes to talk about th...
3    Thank you Florida- a MOVEMENT that has never b...
4    Join me Thursday in Florida &amp; Ohio!West Pa...
Name: TEXT, dtype: object

In [11]:
# top_words=listofwords.value_counts()
# top_words_percent= top_words/len(listofwords)
# top_words.head(50).plot.bar()
# # top_words.head(50)

In [12]:
total_chars=len(tweet_text_all)
total_chars
total_wordz=len((tweet_text_all.split()))
total_wordz

250988

In [13]:
chars = set(tweet_text_all)
words = set(tweet_text_all.split())
print ("total number of unique words", len(words))
print ("total number of unique chars", len(chars))

total number of unique words 32293
total number of unique chars 369


In [14]:
words=list(words)

In [15]:
#replace apostrophes in dictionary keys 
for i in range (len(words)):
    words[i]=words[i].replace("‘", '').replace("’", '').replace("'", '')

In [16]:
words=set(words)
len(words)

31907

In [17]:
#create forward and reverse word index
word_indices = dict((c, i) for i, c in enumerate(words, 1))
indices_word = dict((i, c) for i, c in enumerate(words,1 ))

In [18]:
len(word_indices)
max(word_indices.values())

31907

In [19]:
#choose step 

maxlen = 10
step = 2

sentences = []
next_words = []
next_words = []
list_words = []

sentences2 = []
for i in range (len(tweet_text)):
    list_words = tweet_text.iloc[i].split()
    for i in range(len( list_words)):
        list_words[i]=list_words[i].replace("‘", '').replace("’", '').replace("'", '')
    for i in range(0, len(list_words) - maxlen, step):
        sentences2 = ' '.join(list_words[i: i + maxlen])
        sentences.append(sentences2)
        next_words.append((list_words[i + maxlen]))

print ('length of sentence list:', len(sentences))
print ("length of next_word list", len(next_words))

length of sentence list: 77465
length of next_word list 77465


In [20]:
sequences=[]
y=[]
for i, sentence in enumerate(sentences):
    sequence=[]
    for j, word in enumerate(sentence.split()):
        sequence.append(word_indices[word])
    sequences.append(sequence)
    y.append(word_indices[next_words[i]])

In [21]:
sequences=np.asarray(sequences)
sequences.shape

(77465, 10)

In [22]:
sequences

array([[ 2419, 14037, 10859, ..., 25164, 15720, 12998],
       [10859, 29971,  3155, ..., 12998, 11517, 29971],
       [ 3155,  6145, 16154, ..., 29971, 18074,  5752],
       ...,
       [25164, 19952, 24257, ..., 26230, 30824, 25472],
       [24257, 28566, 13611, ..., 25472, 15970, 29888],
       [13611, 31123, 20271, ..., 29888, 25164,  1248]])

In [23]:
total_words= len(word_indices)+1
total_words

31908

In [24]:
embeddings_index = {}
f = open('../word_embeding/glove.twitter.27B.25d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [25]:
max(word_indices.values())
len(word_indices)

31907

In [26]:
EMBEDDING_DIM=25
embedding_matrix = np.zeros((total_words, EMBEDDING_DIM)) 
for word, i in word_indices.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [27]:
embedding_matrix.shape

(31908, 25)

In [28]:
X=sequences[0:100]
y=y[0:100]

In [29]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Input
from keras.regularizers import L1L2
from keras import regularizers
from keras import metrics
# embedding_layer= Embedding(total_words, EMBEDDING_DIM, weights=[embedding_matrix],input_length=max_seq,trainable=False)
# sequence_input = Input(shape=(max_seq,), dtype='int32')
# embedded_sequences= embedding_layer(sequence_input)
model=Sequential()
# e=Embedding(total_words, EMBEDDING_DIM, weights=[embedding_matrix],input_length=maxlen,trainable=False)
# model.add(e)
from keras.layers import Embedding

model.add( Embedding(total_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            input_shape= (maxlen,),
                            trainable=False))
model.add(LSTM(128, bias_regularizer=regularizers.l1(0.01)))
model.add(Dropout(0.2))
# model.add(LSTM(512, return_sequences=False))
# model.add(Dropout(0.1))
# model.add(Flatten())
model.add(Dense(total_words, activation="softmax"))
optimizer = RMSprop(lr=0.01)
# sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)
model.compile(loss='sparse_categorical_crossentropy',  metrics=['accuracy'], optimizer=optimizer)
model.summary()

# model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model.add(Dense(len(chars), activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer=optimizer)



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 25)            797700    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               78848     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 31908)             4116132   
Total params: 4,992,680
Trainable params: 4,194,980
Non-trainable params: 797,700
_________________________________________________________________


In [36]:
# from keras.models import load_model
# model= load_model("../Saved_models/failed_on_99th_epoch_word_embedding")

In [31]:
# print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_acc', patience=3)
model.fit(X, y, validation_split=0.2, epochs=10,callbacks=[early_stopping])

Instructions for updating:
Use tf.cast instead.
Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 6s 78ms/step - loss: 11.5542 - acc: 0.0125 - val_loss: 10.8645 - val_acc: 0.0500
Epoch 2/10
80/80 [==============================] - 1s 15ms/step - loss: 7.6992 - acc: 0.0250 - val_loss: 10.8853 - val_acc: 0.0000e+00
Epoch 3/10
80/80 [==============================] - 1s 16ms/step - loss: 6.3233 - acc: 0.0375 - val_loss: 10.4412 - val_acc: 0.0500
Epoch 4/10
80/80 [==============================] - 2s 19ms/step - loss: 5.9008 - acc: 0.0375 - val_loss: 10.6911 - val_acc: 0.0500


In [119]:
loss, accuracy = model.evaluate(X_test_sample, y_test_sample, verbose=0)
print('Accuracy: %f' % (accuracy*100))
print('loss: %f' % (loss))
perplexity = np.exp2(loss)
print ('perplexity: {}'.format(perplexity))


Accuracy: 9.900000
loss: 9.635607
perplexity: 795.4386352619334


In [121]:
print (generate_text("i will", 20))

i will be the U.S. and my great job. and the Dems They We including a big deal with Russia and people


In [84]:
# model.save('../failed_on_99th_epoch_word_embedding')

In [37]:
predictions_test=model.predict(X_test)

In [38]:
len(predictions_test)

10639